In [1]:
import dotenv

dotenv.load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(model="gpt-3.5-turbo-1106")

In [3]:
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

chat_history = ChatMessageHistory()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ]
)

chain = prompt | chat 

chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history")

In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings


loader = PyPDFLoader("loan_description.pdf")
pages = loader.load_and_split()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(pages)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever(k=4)

Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)


In [6]:
input = "I want to get a loan"

response = chain_with_history.invoke(
    {"input": input,
     "context": retriever.invoke(input)},
    {"configurable": {"session_id": "unused"}},
)
response

Parent run c4da74d3-001b-4ee0-bda9-ff14af3a5480 not found for run 1e6a1795-9665-4e93-92db-8a361dc29b85. Treating as a root run.


AIMessage(content="For a personal loan, Smart Bank offers competitive rates, simplified documentation, and transparent terms. This type of loan can be used to meet unexpected expenses, plan a dream holiday, cover school or college fees, manage wedding expenditures, fund home improvement expenses, or even purchase consumer durables. If you're interested in Smart Bank's Personal Loan, you can reach out to the Manager at your nearest Smart Bank Branch or contact the Smart Bank Call Centre for more information at 1234.", response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 602, 'total_tokens': 698}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_482d920018', 'finish_reason': 'stop', 'logprobs': None}, id='run-44005c42-cbd2-42b5-940a-e8cf2eb6a8a2-0')